In [7]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
import os
from tqdm import tqdm
from scipy import stats
import seaborn as sns

# ============================
# Constants
# ============================
ROUGHNESS_FILE_PATTERN = "roughness_*.png"  # File pattern to search for
BINS = 500  # Number of bins for histograms
OUTPUT_DIR = "roughness_analysis_output"  # Directory to save output figures
COLOR_ROUGHNESS = 'slategray'
COLOR_GRADIENT_X = 'teal'
COLOR_GRADIENT_Y = 'darkorange'

# ============================
# Functions for processing roughness maps
# ============================
def load_and_normalize(file_path):
    """Load the image file and normalize pixel values to [0,1]."""
    img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise IOError(f"Image not found: {file_path}")
        
    # Handle different image types
    if len(img.shape) == 3:  # Color image
        # Convert to grayscale if it's a color image
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        roughness = img_gray.astype(np.float32) / 255.0
        original_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for display
    else:  # Already grayscale
        roughness = img.astype(np.float32) / 255.0
        original_img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)  # Convert to RGB for display
        
    return original_img, roughness

def compute_gradients(roughness):
    """Compute the gradients of the roughness map."""
    # Compute gradients using Sobel operators
    grad_x = cv2.Sobel(roughness, cv2.CV_32F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(roughness, cv2.CV_32F, 0, 1, ksize=3)
    
    # Normalize gradients to [-1, 1] range
    if np.max(np.abs(grad_x)) > 0:
        grad_x = grad_x / np.max(np.abs(grad_x))
    if np.max(np.abs(grad_y)) > 0:
        grad_y = grad_y / np.max(np.abs(grad_y))
        
    # Compute gradient magnitude
    gradient_mag = np.sqrt(grad_x**2 + grad_y**2)
    if np.max(gradient_mag) > 0:
        gradient_mag = gradient_mag / np.max(gradient_mag)
        
    return grad_x, grad_y, gradient_mag

def compute_statistics(roughness):
    """Compute statistical properties of the roughness map."""
    flat_roughness = roughness.flatten()
    
    # Basic statistics
    mean = np.mean(flat_roughness)
    median = np.median(flat_roughness)
    std = np.std(flat_roughness)
    min_val = np.min(flat_roughness)
    max_val = np.max(flat_roughness)
    
    # Calculate percentiles (10%, 25%, 75%, 90%)
    percentiles = np.percentile(flat_roughness, [10, 25, 75, 90])
    
    # Calculate entropy (measure of randomness)
    hist, _ = np.histogram(flat_roughness, bins=256, range=(0, 1))
    hist = hist / np.sum(hist)  # Normalize
    non_zero_hist = hist[hist > 0]
    entropy = -np.sum(non_zero_hist * np.log2(non_zero_hist)) if len(non_zero_hist) > 0 else 0
    
    # Calculate percentage of areas with different roughness levels
    roughness_areas = {
        'smooth': np.mean(roughness < 0.25) * 100,     # Smooth areas (0-0.25)
        'medium': np.mean((roughness >= 0.25) & (roughness < 0.5)) * 100,  # Medium roughness (0.25-0.5)
        'rough': np.mean((roughness >= 0.5) & (roughness < 0.75)) * 100,   # Rough areas (0.5-0.75)
        'very_rough': np.mean(roughness >= 0.75) * 100  # Very rough areas (0.75-1.0)
    }
    
    # Add texture metrics
    # Skewness (measure of asymmetry)
    skewness = stats.skew(flat_roughness)
    
    # Kurtosis (measure of "tailedness")
    kurtosis = stats.kurtosis(flat_roughness)
    
    # Calculate energy (measure of uniformity)
    energy = np.sum(hist**2)
    
    stats_dict = {
        'mean': mean,
        'median': median,
        'std': std,
        'min': min_val,
        'max': max_val,
        'percentiles': percentiles,
        'entropy': entropy,
        'skewness': skewness,
        'kurtosis': kurtosis,
        'energy': energy,
        'roughness_areas': roughness_areas
    }
    
    return stats_dict

def compute_fourier_analysis(roughness):
    """Compute Fourier transform to analyze frequency components."""
    # Apply FFT
    f_transform = np.fft.fft2(roughness)
    f_shift = np.fft.fftshift(f_transform)
    
    # Calculate the magnitude spectrum (log scale for better visualization)
    magnitude_spectrum = 20 * np.log(np.abs(f_shift) + 1)
    
    # Normalize for display
    if np.max(magnitude_spectrum) > 0:
        normalized_spectrum = magnitude_spectrum / np.max(magnitude_spectrum)
    else:
        normalized_spectrum = magnitude_spectrum
        
    return normalized_spectrum

def process_roughness_map(file_path):
    """Process a single roughness map file."""
    try:
        # Load and normalize the image
        original_img, roughness = load_and_normalize(file_path)
        
        # Compute gradients
        grad_x, grad_y, gradient_mag = compute_gradients(roughness)
        
        # Compute statistics
        stats_dict = compute_statistics(roughness)
        
        # Compute frequency analysis
        frequency_spectrum = compute_fourier_analysis(roughness)
        
        # Create a result dictionary with all the data
        result = {
            'file_path': file_path,
            'original_img': original_img,
            'roughness': roughness,
            'gradient_x': grad_x,
            'gradient_y': grad_y,
            'gradient_mag': gradient_mag,
            'frequency_spectrum': frequency_spectrum,
            'stats': stats_dict
        }
        
        return result
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return None

# ============================
# Functions for plotting
# ============================
def get_global_min_max_y(results, hist_type):
    """Determine global y-axis limits for histograms to ensure consistent scales."""
    max_density = 0
    
    if hist_type == 'roughness':
        for result in results:
            hist, _ = np.histogram(result['roughness'].flatten(), bins=BINS, density=True)
            max_density = max(max_density, np.max(hist))
    elif hist_type == 'gradient':
        for result in results:
            for gradient in [result['gradient_x'], result['gradient_y']]:
                hist, _ = np.histogram(gradient.flatten(), bins=BINS, density=True)
                max_density = max(max_density, np.max(hist))
    
    # Add a small buffer (15%) to the max value for better visualization
    max_density *= 1.15
    
    return (0, max_density)

def plot_roughness_maps(results):
    """Plot each roughness map with its histogram and gradient magnitude."""
    n_images = len(results)
    fig, axs = plt.subplots(n_images, 3, figsize=(18, 6 * n_images))
    if n_images == 1:
        axs = np.expand_dims(axs, axis=0)
        
    # Get global y-axis limits for roughness histograms
    roughness_ylim = get_global_min_max_y(results, 'roughness')
    
    # Calculate max dimensions to ensure consistent zoom levels
    all_heights = []
    all_widths = []
    for result in results:
        h, w, _ = result['original_img'].shape
        all_heights.append(h)
        all_widths.append(w)
    
    max_height = max(all_heights)
    max_width = max(all_widths)
    
    for idx, result in enumerate(results):
        filename = os.path.basename(result['file_path'])
        stats = result['stats']
        
        # Display the roughness map image
        ax_img = axs[idx, 0]
        ax_img.imshow(result['original_img'])
        ax_img.set_title(f"Roughness Map\n{filename}")
        
        # Ensure consistent zoom level across all images
        h, w, _ = result['original_img'].shape
        x_center = w / 2
        y_center = h / 2
        ax_img.set_xlim(x_center - max_width/2, x_center + max_width/2)
        ax_img.set_ylim(y_center + max_height/2, y_center - max_height/2)
        
        ax_img.set_aspect('equal')
        ax_img.axis("off")
        
        # Roughness histogram
        ax_hist = axs[idx, 1]
        sns.histplot(result['roughness'].flatten(), bins=BINS, kde=True, 
                    color=COLOR_ROUGHNESS, edgecolor='black', alpha=0.7, ax=ax_hist)
        ax_hist.set_xlabel("Roughness Value")
        ax_hist.set_ylabel("Probability Density")
        ax_hist.set_title(f"Roughness Distribution - {filename}")
        ax_hist.set_xlim(0, 1)
        ax_hist.set_ylim(*roughness_ylim)
        ax_hist.grid(True, alpha=0.3)
        
        # Add mean and median lines
        ax_hist.axvline(stats['mean'], color='red', linestyle='-', linewidth=2, 
                      label=f'Mean: {stats["mean"]:.3f}')
        ax_hist.axvline(stats['median'], color='green', linestyle='--', linewidth=2, 
                      label=f'Median: {stats["median"]:.3f}')
        
        # Add percentile lines
        ax_hist.axvline(stats['percentiles'][0], color='purple', linestyle=':', linewidth=1,
                      label=f'10th %: {stats["percentiles"][0]:.3f}')
        ax_hist.axvline(stats['percentiles'][3], color='orange', linestyle=':', linewidth=1,
                      label=f'90th %: {stats["percentiles"][3]:.3f}')
        
        ax_hist.legend(loc='upper right', fontsize='small')
        
        # Gradient magnitude visualization
        ax_grad = axs[idx, 2]
        im = ax_grad.imshow(result['gradient_mag'], cmap='hot', vmin=0, vmax=1)
        ax_grad.set_title(f"Gradient Magnitude - {filename}")
        
        # Ensure consistent zoom level for gradient maps too
        ax_grad.set_xlim(x_center - max_width/2, x_center + max_width/2)
        ax_grad.set_ylim(y_center + max_height/2, y_center - max_height/2)
        
        ax_grad.set_aspect('equal')
        ax_grad.axis("off")
        
        # Add colorbar
        plt.colorbar(im, ax=ax_grad, orientation='vertical', label='Gradient Magnitude')
        
    plt.tight_layout()
    return fig

def plot_gradient_histograms(results):
    """Plot histograms of the x and y gradients for each roughness map."""
    n_images = len(results)
    fig, axs = plt.subplots(n_images, 2, figsize=(14, 5 * n_images))
    if n_images == 1:
        axs = np.expand_dims(axs, axis=0)
        
    # Get global y-axis limits for gradient histograms
    gradient_ylim = get_global_min_max_y(results, 'gradient')
    
    for idx, result in enumerate(results):
        filename = os.path.basename(result['file_path'])
        
        # X gradient histogram
        ax_x = axs[idx, 0]
        sns.histplot(result['gradient_x'].flatten(), bins=BINS, kde=True,
                   color=COLOR_GRADIENT_X, edgecolor='black', alpha=0.7, ax=ax_x)
        ax_x.set_xlabel("X Gradient Value")
        ax_x.set_ylabel("Probability Density")
        ax_x.set_title(f"X Gradient Distribution - {filename}")
        ax_x.set_xlim(-1, 1)
        ax_x.set_ylim(*gradient_ylim)
        ax_x.grid(True, alpha=0.3)
        
        # Y gradient histogram
        ax_y = axs[idx, 1]
        sns.histplot(result['gradient_y'].flatten(), bins=BINS, kde=True,
                   color=COLOR_GRADIENT_Y, edgecolor='black', alpha=0.7, ax=ax_y)
        ax_y.set_xlabel("Y Gradient Value")
        ax_y.set_ylabel("Probability Density")
        ax_y.set_title(f"Y Gradient Distribution - {filename}")
        ax_y.set_xlim(-1, 1)
        ax_y.set_ylim(*gradient_ylim)
        ax_y.grid(True, alpha=0.3)
        
    plt.tight_layout()
    return fig

def plot_frequency_analysis(results):
    """Plot frequency domain visualization for each roughness map."""
    n_images = len(results)
    fig, axs = plt.subplots(n_images, 2, figsize=(14, 6 * n_images))
    if n_images == 1:
        axs = np.expand_dims(axs, axis=0)
    
    # Calculate max dimensions for consistent zoom
    all_heights = []
    all_widths = []
    for result in results:
        h, w = result['frequency_spectrum'].shape
        all_heights.append(h)
        all_widths.append(w)
    
    max_height = max(all_heights)
    max_width = max(all_widths)
        
    for idx, result in enumerate(results):
        filename = os.path.basename(result['file_path'])
        
        # Original roughness map
        ax_orig = axs[idx, 0]
        ax_orig.imshow(result['roughness'], cmap='gray')
        ax_orig.set_title(f"Roughness Map - {filename}")
        
        # Ensure consistent zoom
        h, w = result['roughness'].shape
        x_center = w / 2
        y_center = h / 2
        ax_orig.set_xlim(x_center - max_width/2, x_center + max_width/2)
        ax_orig.set_ylim(y_center + max_height/2, y_center - max_height/2)
        
        ax_orig.set_aspect('equal')
        ax_orig.axis("off")
        
        # Frequency spectrum
        ax_freq = axs[idx, 1]
        im = ax_freq.imshow(result['frequency_spectrum'], cmap='viridis')
        ax_freq.set_title(f"Frequency Spectrum - {filename}")
        
        # Ensure consistent zoom for frequency maps too
        h, w = result['frequency_spectrum'].shape
        x_center = w / 2
        y_center = h / 2
        ax_freq.set_xlim(x_center - max_width/2, x_center + max_width/2)
        ax_freq.set_ylim(y_center + max_height/2, y_center - max_height/2)
        
        ax_freq.set_aspect('equal')
        ax_freq.axis("off")
        
        # Add colorbar
        plt.colorbar(im, ax=ax_freq, orientation='vertical', label='Magnitude (dB)')
    
    plt.tight_layout()
    return fig

def plot_roughness_statistics(results):
    """Plot how roughness statistics change per frame."""
    # Sort results by frame number
    # Try to extract the numeric part from the filename
    def extract_number(file_path):
        # Attempt to extract the number from patterns like "roughness_1.png" or "1.png"
        base_name = os.path.basename(file_path)
        name_parts = base_name.split('_')
        
        # If there's an underscore, take the last part, otherwise use the whole filename
        num_part = name_parts[-1] if len(name_parts) > 1 else base_name
        
        # Remove file extension and convert to int
        num_str = os.path.splitext(num_part)[0]
        
        # Try to extract a number, default to 0 if no number found
        try:
            return int(''.join(filter(str.isdigit, num_str)))
        except ValueError:
            return 0
    
    sorted_results = sorted(results, key=lambda r: extract_number(r['file_path']))
    
    frames = list(range(1, len(sorted_results) + 1))
    file_names = [os.path.basename(r['file_path']) for r in sorted_results]
    
    # Extract statistics
    means = [r['stats']['mean'] for r in sorted_results]
    medians = [r['stats']['median'] for r in sorted_results]
    stds = [r['stats']['std'] for r in sorted_results]
    entropies = [r['stats']['entropy'] for r in sorted_results]
    skewness = [r['stats']['skewness'] for r in sorted_results]
    kurtosis = [r['stats']['kurtosis'] for r in sorted_results]
    energy = [r['stats']['energy'] for r in sorted_results]
    
    # Create figure with 4 subplots
    fig, axs = plt.subplots(2, 2, figsize=(16, 12))
    
    # Get proper y-axis ranges for better visualization
    mean_range = max(means) - min(means)
    mean_padding = mean_range * 0.1
    mean_min = max(0, min(means) - mean_padding)
    mean_max = min(1, max(means) + mean_padding)
    
    # Plot mean and median
    ax_central = axs[0, 0]
    ax_central.plot(frames, means, 'o-', color='red', linewidth=2, label='Mean')
    ax_central.plot(frames, medians, 's--', color='green', linewidth=2, label='Median')
    ax_central.set_xlabel("Frame")
    ax_central.set_ylabel("Value")
    ax_central.set_title("Mean and Median Roughness")
    ax_central.set_xticks(frames)
    ax_central.set_xticklabels(file_names, rotation=45)
    ax_central.set_ylim(mean_min, mean_max)
    ax_central.grid(True, linestyle='--', alpha=0.6)
    ax_central.legend()
    
    # Plot standard deviation
    ax_std = axs[0, 1]
    ax_std.plot(frames, stds, 'o-', color='purple', linewidth=2)
    ax_std.set_xlabel("Frame")
    ax_std.set_ylabel("Standard Deviation")
    ax_std.set_title("Roughness Variation (Standard Deviation)")
    ax_std.set_xticks(frames)
    ax_std.set_xticklabels(file_names, rotation=45)
    # Set y-axis limits for better visualization
    std_max = max(stds) * 1.1
    ax_std.set_ylim(0, std_max)
    ax_std.grid(True, linestyle='--', alpha=0.6)
    
    # Plot entropy
    ax_entropy = axs[1, 0]
    ax_entropy.plot(frames, entropies, 'o-', color='blue', linewidth=2)
    ax_entropy.set_xlabel("Frame")
    ax_entropy.set_ylabel("Entropy")
    ax_entropy.set_title("Surface Complexity (Entropy)")
    ax_entropy.set_xticks(frames)
    ax_entropy.set_xticklabels(file_names, rotation=45)
    # Set y-axis limits for better visualization
    entropy_min = min(entropies) * 0.9
    entropy_max = max(entropies) * 1.1
    ax_entropy.set_ylim(entropy_min, entropy_max)
    ax_entropy.grid(True, linestyle='--', alpha=0.6)
    
    # Plot roughness area distribution
    ax_areas = axs[1, 1]
    smooth_areas = [r['stats']['roughness_areas']['smooth'] for r in sorted_results]
    medium_areas = [r['stats']['roughness_areas']['medium'] for r in sorted_results]
    rough_areas = [r['stats']['roughness_areas']['rough'] for r in sorted_results]
    very_rough_areas = [r['stats']['roughness_areas']['very_rough'] for r in sorted_results]
    
    ax_areas.stackplot(frames, 
                     smooth_areas, 
                     medium_areas, 
                     rough_areas, 
                     very_rough_areas,
                     labels=['Smooth (0-0.25)', 'Medium (0.25-0.5)', 'Rough (0.5-0.75)', 'Very Rough (0.75-1.0)'],
                     colors=['skyblue', 'lightgreen', 'orange', 'tomato'],
                     alpha=0.8)
    ax_areas.set_xlabel("Frame")
    ax_areas.set_ylabel("Percentage")
    ax_areas.set_title("Roughness Area Distribution")
    ax_areas.set_xticks(frames)
    ax_areas.set_xticklabels(file_names, rotation=45)
    ax_areas.set_ylim(0, 100)
    ax_areas.grid(True, linestyle='--', alpha=0.6)
    ax_areas.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2)
    
    plt.tight_layout()
    return fig

def plot_advanced_statistics(results):
    """Plot advanced statistical measures across frames."""
    # Sort results same as in plot_roughness_statistics
    def extract_number(file_path):
        base_name = os.path.basename(file_path)
        name_parts = base_name.split('_')
        num_part = name_parts[-1] if len(name_parts) > 1 else base_name
        num_str = os.path.splitext(num_part)[0]
        try:
            return int(''.join(filter(str.isdigit, num_str)))
        except ValueError:
            return 0
    
    sorted_results = sorted(results, key=lambda r: extract_number(r['file_path']))
    
    frames = list(range(1, len(sorted_results) + 1))
    file_names = [os.path.basename(r['file_path']) for r in sorted_results]
    
    # Extract advanced statistics
    skewness = [r['stats']['skewness'] for r in sorted_results]
    kurtosis = [r['stats']['kurtosis'] for r in sorted_results]
    energy = [r['stats']['energy'] for r in sorted_results]
    
    # Create figure with 3 subplots
    fig, axs = plt.subplots(3, 1, figsize=(14, 15))
    
    # Plot skewness (asymmetry)
    ax_skew = axs[0]
    ax_skew.plot(frames, skewness, 'o-', color='teal', linewidth=2)
    ax_skew.set_xlabel("Frame")
    ax_skew.set_ylabel("Skewness")
    ax_skew.set_title("Distribution Asymmetry (Skewness)")
    ax_skew.set_xticks(frames)
    ax_skew.set_xticklabels(file_names, rotation=45)
    ax_skew.axhline(y=0, color='gray', linestyle='--', alpha=0.6)
    ax_skew.grid(True, linestyle='--', alpha=0.6)
    # Add annotation about meaning
    ax_skew.annotate('Positive: More rough areas\nNegative: More smooth areas', 
                    xy=(0.02, 0.85), xycoords='axes fraction',
                    bbox=dict(boxstyle="round,pad=0.3", fc="lightyellow", ec="orange", alpha=0.8))
    
    # Plot kurtosis (peakedness)
    ax_kurt = axs[1]
    ax_kurt.plot(frames, kurtosis, 'o-', color='purple', linewidth=2)
    ax_kurt.set_xlabel("Frame")
    ax_kurt.set_ylabel("Kurtosis")
    ax_kurt.set_title("Distribution Peakedness (Kurtosis)")
    ax_kurt.set_xticks(frames)
    ax_kurt.set_xticklabels(file_names, rotation=45)
    ax_kurt.axhline(y=0, color='gray', linestyle='--', alpha=0.6)
    ax_kurt.grid(True, linestyle='--', alpha=0.6)
    # Add annotation about meaning
    ax_kurt.annotate('Positive: More extreme values\nNegative: More uniform distribution', 
                    xy=(0.02, 0.85), xycoords='axes fraction',
                    bbox=dict(boxstyle="round,pad=0.3", fc="lightyellow", ec="orange", alpha=0.8))
    
    # Plot energy (uniformity)
    ax_energy = axs[2]
    ax_energy.plot(frames, energy, 'o-', color='darkred', linewidth=2)
    ax_energy.set_xlabel("Frame")
    ax_energy.set_ylabel("Energy")
    ax_energy.set_title("Texture Uniformity (Energy)")
    ax_energy.set_xticks(frames)
    ax_energy.set_xticklabels(file_names, rotation=45)
    ax_energy.grid(True, linestyle='--', alpha=0.6)
    # Add annotation about meaning
    ax_energy.annotate('Higher: More uniform texture\nLower: More varied texture', 
                     xy=(0.02, 0.85), xycoords='axes fraction',
                     bbox=dict(boxstyle="round,pad=0.3", fc="lightyellow", ec="orange", alpha=0.8))
    
    plt.tight_layout()
    return fig

def plot_comparative_analysis(results):
    """Create a comparative visualization of all roughness maps side by side."""
    # Sort results
    def extract_number(file_path):
        base_name = os.path.basename(file_path)
        name_parts = base_name.split('_')
        num_part = name_parts[-1] if len(name_parts) > 1 else base_name
        num_str = os.path.splitext(num_part)[0]
        try:
            return int(''.join(filter(str.isdigit, num_str)))
        except ValueError:
            return 0
    
    sorted_results = sorted(results, key=lambda r: extract_number(r['file_path']))
    
    # Calculate the grid layout (try to make it square-ish)
    n_images = len(sorted_results)
    grid_size = int(np.ceil(np.sqrt(n_images)))
    rows = grid_size
    cols = grid_size
    
    # Create a large figure for the comparison
    fig, axs = plt.subplots(rows, cols, figsize=(cols*4, rows*4))
    
    # Calculate global min/max for colormap normalization
    all_roughness = np.concatenate([r['roughness'].flatten() for r in sorted_results])
    vmin, vmax = np.min(all_roughness), np.max(all_roughness)
    
    # Calculate max dimensions to ensure consistent zoom levels
    all_heights = []
    all_widths = []
    for result in sorted_results:
        h, w = result['roughness'].shape
        all_heights.append(h)
        all_widths.append(w)
    
    max_height = max(all_heights)
    max_width = max(all_widths)
    
    # Make sure axs is always a 2D array
    if rows == 1 and cols == 1:
        axs = np.array([[axs]])
    elif rows == 1:
        axs = axs.reshape(1, -1)
    elif cols == 1:
        axs = axs.reshape(-1, 1)
    
    # Plot each roughness map with consistent colormap
    for idx, result in enumerate(sorted_results):
        if idx < rows * cols:
            row = idx // cols
            col = idx % cols
            
            ax = axs[row, col]
            filename = os.path.basename(result['file_path'])
            
            # Plot the roughness map
            im = ax.imshow(result['roughness'], cmap='viridis', vmin=vmin, vmax=vmax)
            
            # Ensure consistent zoom level
            h, w = result['roughness'].shape
            x_center = w / 2
            y_center = h / 2
            ax.set_xlim(x_center - max_width/2, x_center + max_width/2)
            ax.set_ylim(y_center + max_height/2, y_center - max_height/2)
            
            ax.set_title(f"{filename}\nMean: {result['stats']['mean']:.3f}")
            ax.axis('off')
            
    # Turn off any unused subplots
    for idx in range(n_images, rows*cols):
        row = idx // cols
        col = idx % cols
        axs[row, col].axis('off')
    
    # Add a colorbar at the bottom
    cbar_ax = fig.add_axes([0.15, 0.07, 0.7, 0.02])
    cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
    cbar.set_label('Roughness Value')
    
    plt.suptitle('Comparative Analysis of Roughness Maps', fontsize=20, y=0.98)
    plt.tight_layout(rect=[0, 0.1, 1, 0.95])  # Adjust layout for the colorbar
    return fig

# ============================
# Main function
# ============================
def analyze_roughness_maps(file_pattern=ROUGHNESS_FILE_PATTERN, output_dir=OUTPUT_DIR):
    """Analyze roughness texture maps."""
    print("Starting roughness texture map analysis...")
    
    # Find all files matching the roughness file pattern
    roughness_files = glob.glob(file_pattern)
    if len(roughness_files) < 1:
        print(f"No roughness map images found matching pattern '{file_pattern}'.")
        return
    
    # Sort the files numerically
    roughness_files = sorted(roughness_files, 
                           key=lambda f: int(''.join(filter(str.isdigit, os.path.splitext(os.path.basename(f))[0])) or 0))
    
    print(f"Found {len(roughness_files)} roughness map files: {roughness_files}")
    
    # Process each roughness map file with progress bar
    results = []
    for file in tqdm(roughness_files, desc="Processing roughness maps"):
        try:
            result = process_roughness_map(file)
            if result is not None:  # Only add valid results
                results.append(result)
                
                # Print a summary of the statistics
                stats = result['stats']
                print(f"\n  {file} Statistics:")
                print(f"    - Mean: {stats['mean']:.4f}")
                print(f"    - Median: {stats['median']:.4f}")
                print(f"    - Std Dev: {stats['std']:.4f}")
                print(f"    - Entropy: {stats['entropy']:.4f}")
                print(f"    - Skewness: {stats['skewness']:.4f}")
                print(f"    - Kurtosis: {stats['kurtosis']:.4f}")
                print(f"    - Area %: Smooth={stats['roughness_areas']['smooth']:.1f}%, "
                      f"Medium={stats['roughness_areas']['medium']:.1f}%, "
                      f"Rough={stats['roughness_areas']['rough']:.1f}%, "
                      f"Very Rough={stats['roughness_areas']['very_rough']:.1f}%")
        except Exception as e:
            print(f"Error in main loop processing {file}: {e}")
    
    if not results:
        print("No valid roughness maps to process.")
        return
    
    print(f"\nSuccessfully processed {len(results)} roughness maps. Generating figures...")
    
    # Create a directory for saving figures
    os.makedirs(output_dir, exist_ok=True)
    
    # Set figure saving parameters
    plt.rcParams['savefig.dpi'] = 300
    plt.rcParams['figure.autolayout'] = True
    
    # Figure 1: Plot roughness maps with histograms
    fig1 = plot_roughness_maps(results)
    fig1.savefig(os.path.join(output_dir, "figure1_roughness_maps.png"))
    plt.close(fig1)
    
    # Figure 2: Plot gradient histograms
    fig2 = plot_gradient_histograms(results)
    fig2.savefig(os.path.join(output_dir, "figure2_gradient_histograms.png"))
    plt.close(fig2)
    
    # Figure 3: Plot frequency analysis
    fig3 = plot_frequency_analysis(results)
    fig3.savefig(os.path.join(output_dir, "figure3_frequency_analysis.png"))
    plt.close(fig3)
    
    # Figure 4: Plot roughness statistics
    fig4 = plot_roughness_statistics(results)
    fig4.savefig(os.path.join(output_dir, "figure4_roughness_statistics.png"))
    plt.close(fig4)
    
    # Figure 5: Plot advanced statistics
    fig5 = plot_advanced_statistics(results)
    fig5.savefig(os.path.join(output_dir, "figure5_advanced_statistics.png"))
    plt.close(fig5)
    
    # Figure 6: Plot comparative analysis
    fig6 = plot_comparative_analysis(results)
    fig6.savefig(os.path.join(output_dir, "figure6_comparative_analysis.png"))
    plt.close(fig6)
    
    print(f"\nAll figures have been saved to the '{output_dir}' directory.")
    
    # Create a summary report with key metrics
    create_summary_report(results, output_dir)
    
    return results

def create_summary_report(results, output_dir):
    """Create a summary report of the analysis."""
    summary_file = os.path.join(output_dir, "roughness_analysis_summary.txt")
    
    with open(summary_file, 'w') as f:
        f.write("Roughness Analysis Summary Report\n")
        f.write("=" * 40 + "\n")
        for result in results:
            filename = os.path.basename(result['file_path'])
            stats = result['stats']
            
            f.write(f"\nFile: {filename}\n")
            f.write(f"Mean: {stats['mean']:.4f}\n")
            f.write(f"Median: {stats['median']:.4f}\n")
            f.write(f"Standard Deviation: {stats['std']:.4f}\n")
            f.write(f"Entropy: {stats['entropy']:.4f}\n")
            f.write(f"Skewness: {stats['skewness']:.4f}\n")
            f.write(f"Kurtosis: {stats['kurtosis']:.4f}\n")
            f.write(f"Energy: {stats['energy']:.4f}\n")
            f.write(f"Area %: Smooth={stats['roughness_areas']['smooth']:.1f}%, "
                    f"Medium={stats['roughness_areas']['medium']:.1f}%, "
                    f"Rough={stats['roughness_areas']['rough']:.1f}%, "
                    f"Very Rough={stats['roughness_areas']['very_rough']:.1f}%\n")
    
    print(f"\nSummary report saved to '{summary_file}'.")

    print("Analysis complete!")

if __name__ == "__main__":
    # Run the analysis with default parameters
    analyze_roughness_maps()

Starting roughness texture map analysis...
Found 3 roughness map files: ['roughness_1.png', 'roughness_2.png', 'roughness_3.png']


Processing roughness maps:  33%|███▎      | 1/3 [00:04<00:09,  4.73s/it]


  roughness_1.png Statistics:
    - Mean: 0.0168
    - Median: 0.0118
    - Std Dev: 0.0179
    - Entropy: 3.6851
    - Skewness: 3.1516
    - Kurtosis: 29.5216
    - Area %: Smooth=100.0%, Medium=0.0%, Rough=0.0%, Very Rough=0.0%


Processing roughness maps:  67%|██████▋   | 2/3 [00:09<00:04,  4.66s/it]


  roughness_2.png Statistics:
    - Mean: 0.0200
    - Median: 0.0157
    - Std Dev: 0.0209
    - Entropy: 3.9063
    - Skewness: 3.1881
    - Kurtosis: 29.5187
    - Area %: Smooth=100.0%, Medium=0.0%, Rough=0.0%, Very Rough=0.0%


Processing roughness maps: 100%|██████████| 3/3 [00:13<00:00,  4.65s/it]


  roughness_3.png Statistics:
    - Mean: 0.0275
    - Median: 0.0196
    - Std Dev: 0.0291
    - Entropy: 4.3368
    - Skewness: 2.7726
    - Kurtosis: 19.1373
    - Area %: Smooth=100.0%, Medium=0.0%, Rough=0.0%, Very Rough=0.0%

Successfully processed 3 roughness maps. Generating figures...



C:\Users\antoi\AppData\Local\Temp\ipykernel_35352\1703057997.py:628: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0.1, 1, 0.95])  # Adjust layout for the colorbar



All figures have been saved to the 'roughness_analysis_output' directory.

Summary report saved to 'roughness_analysis_output\roughness_analysis_summary.txt'.
Analysis complete!
